# The Ames Housing Dataset

The Ames Housing Dataset(the file Ames_HousePrice) has data for 2580 house sales, there are 81 descriptive variables for each house. We have already (in the Ames_ML_RealEstate_Preprocessing notebook) created the 'ames final' file, which merges Ames_HousePrice with the Real Estate file, gaining the 7 new variables GeoRefNo, lat, long, prop_addr, MAzip1, distance to ISU, and distance category. We have already dropped PID and GeoRefNo, so in the ames_final file there are 86 features.  

In [1]:
import pandas as pd
from datetime import date

In [2]:
housing = pd.read_csv('data/ames_final.csv')

In [3]:
print("Number of houses: ", housing['SalePrice'].count())
print("Number of features in dataset: ", len(housing.columns))
#print("Columns in dataset: ", housing.columns)

Number of houses:  2579
Number of features in dataset:  87


# Missing Data

In [4]:
# Handling these duplicates is no longer necessary, 
#   its done in the real estate preprocessing

#drop these duplicates
#print(len(housing.PID.unique()))
#print(len(housing))

#duplicates = housing.duplicated()
#print("Number of duplicates: ", len(housing[duplicates]))
#housing = housing.drop_duplicates()

#print(len(housing))

In [5]:
# Here we look at variables that need to be removed because of missing values.
# For now, we will leave all of these in, to see if they add value to the model

missing_values=housing.isna().sum()
missing_values_features=missing_values[missing_values>0].sort_values(ascending=False)
missing_values_features
#missing_values_features.plot(kind='bar', figsize=(10,6))

PoolQC          2570
MiscFeature     2482
Alley           2411
Fence           2054
FireplaceQu     1241
LotFrontage      462
GarageCond       129
GarageQual       129
GarageFinish     129
GarageYrBlt      129
GarageType       127
BsmtExposure      71
BsmtFinType2      70
BsmtQual          69
BsmtCond          69
BsmtFinType1      69
MA_Zip1           22
Prop_Addr         20
MasVnrArea        14
MasVnrType        14
BsmtHalfBath       2
BsmtFullBath       2
Electrical         1
TotalBsmtSF        1
GarageCars         1
GarageArea         1
BsmtUnfSF          1
BsmtFinSF2         1
BsmtFinSF1         1
dtype: int64

In [6]:
#Here we see that PoolQC, MiscFeature, Alley, and Fence 
#  all have mostly missing values, so drop them from the housing dataset. Now there are 77  features.
#For now, we leave them in to explore the model. 
#housing=housing.drop(columns=['PoolQC', 'MiscFeature', 'Alley', 'Fence'])

In [7]:
#housing.info()
#housing.describe()
#housing.head()
#housing.columns

# Data Cleaning

In [8]:
#housing[housing['MSZoning'].isin(['I (all)'])]

In [9]:
# Drop I, C, A (industrial, commercial, agri)
# Found 1 row with A, 17 with C, 2 with I
# https://www.zoneomics.com/zoning-maps/iowa/ames
#    Commented out because these houses are still residential, just in A,C,I zones

#print(housing.groupby('MSZoning').count())

#housing = housing[housing['MSZoning'].isin(['FV','RH','RL','RM'])]

#print(housing['MSZoning'].unique())
#print(housing['MSZoning'].count())

In [10]:
# 2 rows with gps info but missing zip:
specified_addresses = ['2010 KILDEE ST', '1310 WOODSTOCK AVE']
# Update 'Zipcode' to 50014 where 'Address' is in the specified list
housing.loc[housing['Prop_Addr'].isin(specified_addresses), 'MA_Zip1'] = 50014
    
# 'Prop_Addr' column useful for EDA and data cleaning but not for modeling
housing.drop('Prop_Addr',axis=1,inplace=True)

# Reset index after dropping rows so flitering by iloc works smoothly
housing.reset_index(drop=True,inplace=True)

# Delete utilities column, it has 2496/2497 with same value
    # housing.Utilities.value_counts() # <-- No N/As here
housing.drop('Utilities',axis=1,inplace=True)

# Fix Row with Missing Values in Basement Categories
housing.loc[housing.PID==903230120,[
    'BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF',
    'BsmtFullBath','BsmtHalfBath']] = housing.loc[housing.PID==903230120,[
    'BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF','BsmtFullBath','BsmtHalfBath']].fillna(0)

# Masonry Veneer (Type/Area)
    # If Type = 'None' but Area != 0, Type updated to mode
    # N/As updated to None for Type and 0 for Area
idx = (housing['MasVnrArea'].isna()) & (housing['MasVnrType'].isna())
housing.loc[idx, 'MasVnrArea'] = housing.loc[idx, 'MasVnrArea'].fillna(0)
housing.loc[idx, 'MasVnrType'] = housing.loc[idx, 'MasVnrType'].fillna('None')
mode_MasVnrType = housing.loc[housing.MasVnrType!='None'].MasVnrType.mode()[0]
idx2 = (housing['MasVnrArea']!=0) & (housing['MasVnrType']== 'None')
housing.loc[idx2, 'MasVnrType'] = housing.loc[idx2, 'MasVnrType'].fillna(mode_MasVnrType)

# Fix Row with Missing Values in GarageCars & GarageArea Categories
idx3 = (housing['GarageArea']!=0) & (housing['GarageType']=='Detchd')
mean_GarageArea = round(housing.loc[idx3, 'GarageArea'].mean())
housing.loc[housing.PID==910201180,'GarageArea'] = mean_GarageArea
idx4 = housing['GarageType']=='Detchd'
mode_GarageCars = housing.loc[idx4, 'GarageCars'].mode()[0]
housing.loc[housing.PID==910201180,'GarageCars'] = mode_GarageCars

# Fill GarageYrBlt to match Year House Was Built
housing['GarageYrBlt'] = housing['GarageYrBlt'].fillna(housing['YearBuilt']) # 129 N/A

# Fix Row where Year Remodeled occured before Year Built
housing.loc[housing.PID==907194160,'YearRemodAdd'] = \
housing.loc[housing.PID==907194160,'YearBuilt']

# Fill with Most Common (Categorical) Value:
mode_Electrical = housing['Electrical'].mode()[0] # 1 N/A
housing['Electrical'].fillna(mode_Electrical, inplace=True)

# 451 N/A values for Lot Frontage
    # The average ratio of LotFrontage/LotArea was calculated for each
    # LotType & LotShape and this percentage factor was used to calculate
    # The missing LotFrontage Values (since LotArea was not missing)
lot_df = housing.loc[housing['LotFrontage'].notna()][[
    'LotFrontage','LotArea','LotConfig','LotShape']]
lot_df2 = lot_df.groupby(['LotConfig','LotShape']).agg({'LotFrontage':'mean','LotArea':'mean'})
lot_df2['PCT_Frontage']=lot_df2['LotFrontage']/lot_df2['LotArea']
percentage_factor_mapping = lot_df2['PCT_Frontage'].to_dict()

# Update the NaN values in 'LotFrontage' using the mapping and LotArea
housing['LotFrontage'] = housing.apply(
    lambda row: round(row['LotArea'] * percentage_factor_mapping.get(
        (row['LotConfig'], row['LotShape']), 1.0)), axis=1)



In [11]:
# Remove outliers 528351010 has too high sale price, 908154205, has too much area
housing = housing[~housing['PID'].isin([528351010, 908154205])]

# Drop PID, only needed for merging
housing.drop(['PID'],axis=1,inplace=True)

# Replace MSSubClass numerical values with letter values since it is categorical
MSSubClass_mapping_dict = {20:'A', 30:'B', 40:'C', 45:'D', 50:'E', 60:'F', 70:'G', 75:'H', 
                           80:'I', 85:'J', 90:'K', 120:'L', 150:'M', 160:'N', 180:'O', 190:'P'}
housing['MSSubClass'] = housing['MSSubClass'].replace(MSSubClass_mapping_dict)

# Update Zip Code Column.  There are only a few zips listed actually in Ames. The rest are
    # changed to 'other' rather than search the address and update to the appropriate one.
    # The column is also treated as a categorical variable
housing['MA_Zip1'].fillna(99, inplace=True)
housing['MA_Zip1'] = housing['MA_Zip1'].astype(int)
housing['MA_Zip1'] = housing['MA_Zip1'].astype(str)
zip_update = ~housing['MA_Zip1'].isin(['50010', '50011', '50012', '50013', '50014'])
housing.loc[zip_update, 'MA_Zip1'] = 'other'


# Adding Features

In [12]:
# Calculate the YearsSinceRemod column
housing['YearsSinceRemod'] = housing['YearRemodAdd'] - housing['YearBuilt']
# Change YearBuilt to Age so it works better as a numerical variable
housing['Age'] = housing['YrSold'] - housing['YearBuilt']
housing.drop('YearBuilt',axis=1,inplace=True)

# Create the 'TotalSF' feature
housing['TotalSF'] = housing.GrLivArea + housing.TotalBsmtSF

#This feature causes data leakage if included in the modeling step
#housing['PricePerSF'] = housing['SalePrice'] / housing['GrLivArea']

date_sold = list(zip(housing['YrSold'], housing['MoSold']))
housing['DateSold'] = [date(y,m,15) for y,m in date_sold]

# Combine Full and Half Bathroom Categories
housing['BsmtBath']=housing['BsmtFullBath']+0.5*housing['BsmtHalfBath']
housing['Bath']=housing['FullBath']+0.5*housing['HalfBath']
housing.drop(['BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath'],axis=1,inplace=True)
mean_BsmtBath = round(housing.BsmtBath.mean()*2)/2 # Round to nearest 0.5
housing['BsmtBath'].fillna(mean_BsmtBath, inplace=True)

# Replace TotRmsAbvGrd with MiscRmsAbvGrd to avoid multicollinearity?
    # These are not bedroom, kitchen, nor bathroom
housing['MiscRmsAbvGrd'] = housing.TotRmsAbvGrd - housing.BedroomAbvGr - housing.KitchenAbvGr
housing.drop('TotRmsAbvGrd',axis=1,inplace=True)

print(f'Shape of DataFrame: {housing.shape}')

Shape of DataFrame: (2577, 85)


# Mapping Data for Ordinal Categorical Values

In [13]:
dict_map5 = {'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5}
dict_map05 = {'None': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5}
dict_map04 = {'None': 0, 'Fa': 1, 'TA': 2, 'Gd': 3, 'Ex': 4}
dict_map_slope = {'Sev': 1, 'Mod': 2, 'Gtl': 3}
housing['ExterQual']=housing['ExterQual'].map(dict_map5)
housing['ExterCond']=housing['ExterCond'].map(dict_map5)
housing['BsmtQual']=housing['BsmtQual'].map(dict_map05)
housing['BsmtCond']=housing['BsmtCond'].map(dict_map05)
housing['HeatingQC']=housing['HeatingQC'].map(dict_map5)
housing['KitchenQual']=housing['KitchenQual'].map(dict_map5)
housing['FireplaceQu']=housing['FireplaceQu'].map(dict_map05)
housing['GarageQual']=housing['GarageQual'].map(dict_map05)
housing['GarageCond']=housing['GarageCond'].map(dict_map05)
housing['PoolQC']=housing['PoolQC'].map(dict_map04)
housing['LandSlope']=housing['LandSlope'].map(dict_map_slope)

# Sorting Features Into Numerical/Categorical

In [14]:
# Loop through each feature and examine datatype
numerical_features = []
cat_ord_features = [] #ordinal features, can be ranked from hi to lo
cat_nom_features = [] #nominal features, cannot be ranked

for i, feature in enumerate(housing.columns):
    if feature=='PoolArea' or feature=='DateSold' or feature=='YrSold':
        numerical_features.append(feature)
    elif feature=='MSSubClass' or feature=='MoSold':
        cat_nom_features.append(feature)
    elif feature=='LandSlope':
        cat_ord_features.append(feature)
    elif housing[feature].dtype=='object':
        cat_nom_features.append(feature)
    else:
        if len(housing[feature].unique()) < 20:
            cat_ord_features.append(feature)
        else:
            numerical_features.append(feature)

print("Number of numerical features: ", len(numerical_features))
for i, feature in enumerate(housing[numerical_features].columns):
    print("Numerical\t", feature, "\t Number of unique values: ", len(housing[feature].unique()))

print("Number of ordinal features: ", len(cat_ord_features))
for i, feature in enumerate(housing[cat_ord_features].columns):
    print("Categorical, ordinal\t\t", feature, housing[feature].unique())

print("Number of nominal features: ", len(cat_nom_features))
for i, feature in enumerate(housing[cat_nom_features].columns):
    print("Categorical, nom\t\t", feature, housing[feature].unique())

tot=len(numerical_features)+len(cat_ord_features)+len(cat_nom_features)
print("Total number of features: ", tot)

Number of numerical features:  30
Numerical	 GrLivArea 	 Number of unique values:  1211
Numerical	 SalePrice 	 Number of unique values:  868
Numerical	 LotFrontage 	 Number of unique values:  184
Numerical	 LotArea 	 Number of unique values:  1751
Numerical	 YearRemodAdd 	 Number of unique values:  61
Numerical	 MasVnrArea 	 Number of unique values:  414
Numerical	 BsmtFinSF1 	 Number of unique values:  925
Numerical	 BsmtFinSF2 	 Number of unique values:  264
Numerical	 BsmtUnfSF 	 Number of unique values:  1059
Numerical	 TotalBsmtSF 	 Number of unique values:  987
Numerical	 1stFlrSF 	 Number of unique values:  1024
Numerical	 2ndFlrSF 	 Number of unique values:  590
Numerical	 LowQualFinSF 	 Number of unique values:  30
Numerical	 GarageYrBlt 	 Number of unique values:  107
Numerical	 GarageArea 	 Number of unique values:  572
Numerical	 WoodDeckSF 	 Number of unique values:  372
Numerical	 OpenPorchSF 	 Number of unique values:  238
Numerical	 EnclosedPorch 	 Number of unique valu

In [15]:
#numerical_features
#cat_ord_features
#cat_nom_features

In [16]:
housing.to_csv('data/housing_with_cats.csv', index=False)

In [17]:
# this is manual label encoding, there is a function in sklearn to do this
# LabelEncoder()
df_numerical = housing.copy()
for feature in cat_nom_features:
    col_map = {}
    for i in range(len(df_numerical[feature].unique())):
        col_map[list(df_numerical[feature].unique())[i]] = i+1
    df_numerical[feature] = df_numerical[feature].map(col_map)


In [18]:
df_numerical.to_csv('data/housing_numerical.csv', index=False)